In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier



url = '/Users/Roby/MLFieldWork/DF_Model.csv'
target = 'target'
variables = ['client_create_date', 'region', 'trade_sector', 'n_employees', 'economic_pot', 'eco_pot_class', 'risk_cat', 'flg_tool', 'sales_channel', 'net', 'unit', 'family_code', 'group_code', 'n_purchases', 'sales_net']


In [4]:
df = pd.read_csv(url)
df.head()

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,family_code,group_code,sales_id,n_purchases,sales_net,time_diff,target
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2,1579.03,NaN,1
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2,1341.14,1126.0,1
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,0,B,20.02,XBSI4AA,XBSI4AA0101,8321_201804,7,20.02,NaN,1


In [5]:
df.columns

Index(['client_id', 'client_create_date', 'region', 'trade_sector',
       'n_employees', 'economic_pot', 'eco_pot_class', 'risk_cat', 'yyyymm',
       'item_id', 'flg_tool', 'sales_channel', 'net', 'family_code',
       'group_code', 'sales_id', 'n_purchases', 'sales_net', 'time_diff',
       'target'],
      dtype='object')

In [6]:
df[['client_id', 'item_id', 'sales_id', 'trade_sector']] = df[['client_id', 'item_id', 'sales_id', 'trade_sector']].astype(str)

df

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,family_code,group_code,sales_id,n_purchases,sales_net,time_diff,target
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2,1579.03,NaN,1
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN,1
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2,1341.14,1126.0,1
4,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,0,B,20.02,XBSI4AA,XBSI4AA0101,8321_201804,7,20.02,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416525,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,12174,1,B,0.00,XADC1AA,XADC1AA0401,93244_202112,1,7365.95,NaN,0
416526,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,2841,0,B,331.40,XADD2EB,XADD2EB0201,93244_202112,1,7365.95,NaN,0
416527,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,5292,1,B,2298.99,XADD1BA,XADD1BA0602,93244_202112,1,7365.95,NaN,0
416528,93244,2021-12-23,RE,11000,1,1509.51,E,5d,2021-12-01,5282,1,B,1399.00,XADD1BB,XADD1BB0401,93244_202112,1,7365.95,NaN,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416530 entries, 0 to 416529
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   client_id           416530 non-null  object 
 1   client_create_date  416530 non-null  object 
 2   region              407750 non-null  object 
 3   trade_sector        416530 non-null  object 
 4   n_employees         416530 non-null  int64  
 5   economic_pot        416530 non-null  float64
 6   eco_pot_class       416530 non-null  object 
 7   risk_cat            416530 non-null  object 
 8   yyyymm              416530 non-null  object 
 9   item_id             416530 non-null  object 
 10  flg_tool            416530 non-null  int64  
 11  sales_channel       416530 non-null  object 
 12  net                 416530 non-null  float64
 13  family_code         416530 non-null  object 
 14  group_code          416530 non-null  object 
 15  sales_id            416530 non-nul

In [8]:
# Select numerical columns for SelectKBest
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.drop(['target', 'time_diff'])

# Apply SelectKBest
X = df[numerical_features]
y = df[target]


selector = SelectKBest(score_func=f_classif, k='all')  # Select all features for ranking
X_selected = selector.fit(X, y)

# Show feature scores
feature_scores = pd.DataFrame({'Feature': numerical_features, 'Score': X_selected.scores_})
print(feature_scores.sort_values(by='Score', ascending=False))

        Feature         Score
4   n_purchases  81680.766060
1  economic_pot   7461.446014
5     sales_net   1832.614432
2      flg_tool     99.735120
0   n_employees     37.189757
3           net     17.487671


In [9]:
# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
X = df[variables]
y = df[target]
rf.fit(X, y)

# Get feature importances
feature_importance = pd.DataFrame({'Feature': numerical_features, 'Importance': rf.feature_importances_})
print(feature_importance.sort_values(by='Importance', ascending=False))

KeyError: "['unit'] not in index"